
In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Transform data to numerical
# 0 is a smoker, 1 is not
# 0 is a female, 1 is a male
# for region, 0 is southwest, 1 is southeast, 2 is northwest, 3 is northeast
dataset['smoker'] = np.where(dataset['smoker'] == 'no', 0, 1)
dataset['sex'] = np.where(dataset['sex'] == 'female', 0, 1)
dataset['region'] = pd.factorize(dataset['region'])[0]
dataset.tail()

In [ ]:
X = dataset.pop('expenses')
Y = dataset

In [ ]:
# Split the data set
train_dataset, test_dataset, train_labels, test_labels = train_test_split(Y, X, test_size=0.2, train_size=0.8, random_state=2)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_dataset = sc.fit_transform(train_dataset)
test_dataset = sc.transform(test_dataset)

In [ ]:
normalizer= tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))
first = np.array(train_dataset[:1])

model = tf.keras.Sequential([
                             normalizer,
                             layers.Dense(units=1)
])
model.add(tf.keras.layers.Dense(units = 32, activation='relu'))
model.add(tf.keras.layers.Dense(units = 16, activation='relu'))
model.add(tf.keras.layers.Dense(units = 8, activation='relu'))
model.add(tf.keras.layers.Dense(units = 4, activation='relu'))
model.add(tf.keras.layers.Dense(units = 2, activation='relu'))
model.add(tf.keras.layers.Dense(units = 1))
model.summary() 

In [ ]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae', 'mse'])

model.fit(x=train_dataset, y=train_labels, epochs=100, verbose=1)

In [ ]:
model.evaluate(test_dataset, test_labels)
model.predict(test_dataset[:5])

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
